In [25]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import json
import requests

In [26]:
def input_data(driver,class_O): #driver과 필터값을 주면 해당필터의 목록을 보여주고
                                #값을 받아 해당 카테고리로 이동하는 함수
    main_url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&includeMen=false&priceMin=&priceMax="
    
    
    if class_O == "기간":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[1]/dl/dd/ul"""
    elif class_O == "대분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[2]/dl/dd/ul"""
    elif class_O == "중분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[3]/dl/dd/ul"""
    elif class_O == "가격":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[4]/dl/dd/ul"""
    else:
        print("잘못된 input_data함수 사용")
        return 0
    
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.TAG_NAME,"a")
    print("입력가능 항목")
    word = []

    for date in data_list:
        word.append(date.get_attribute("innerText"))
    print(word)
    input_date = input("원하시는 항목을 입력해 주십시오\n")

    try:
        if input_date in word:

            data_list[word.index(input_date)].click()
            
        else:
            raise TypeError 
    except TypeError:
        print("입력가능한 항목이 아닙니다.")
    
def crawling_data(driver):
    xpath = """//*[@id="goodsRankList"]"""
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.CLASS_NAME,"article_info")







In [48]:
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":"/home/rds/amr_ws/edm/data",
        "download.prompt_for_download":False}
options.add_experimental_option("prefs",prefs)
url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="
driver = webdriver.Chrome(service=Service("../../driver/chromedriver-linux64/chromedriver"),
                        options=options)
driver.get(url)
driver.maximize_window()

#직접 유저가 넣는것에서 바뀌면 사이에 대기시간이 요구될 수 있음
input_data(driver, "기간")
input_data(driver, "대분류")
input_data(driver,"중분류")
input_data(driver, "가격")








입력가능 항목
['실시간', '일간', '주간', '월간', '3개월']
입력가능 항목
['상의', '아우터', '바지', '원피스', '스커트', '스니커즈', '신발', '가방', '여성 가방', '스포츠/용품', '모자', '양말/레그웨어', '속옷', '선글라스/안경테', '액세서리', '시계', '주얼리', '뷰티', '디지털/테크', '리빙', '컬처', '반려동물']
입력가능 항목
['데님 팬츠', '코튼 팬츠', '슈트 팬츠/슬랙스', '트레이닝/조거 팬츠', '숏 팬츠', '레깅스', '점프 슈트/오버올', '스포츠 하의', '기타 바지']
입력가능 항목
['5만원 이하', '5~10만원', '10~20만원', '20~30만원', '30만원 이상']


In [50]:
####################

xpath = """//*[@id="goodsRankList"]"""
data_bar = driver.find_element(By.XPATH, xpath)
data_list = data_bar.find_elements(By.CLASS_NAME,"list_img")
product_link = []
product_names = []
product_brand = []
product_gender = []
product_sell = []
product_goods = []
product_review_count = []
product_member_price = []
product_nomal_price = []
product_tags = []
product_bigCategory = []
product_smallCategory = []
product_numbers=[]
dd=[]
goods_tmp = driver.find_element(By.ID,"goodsRankList")
goods_like = goods_tmp.find_elements(By.NAME,"count")

for like in goods_like:
    product_goods.append(like.get_attribute("innerText"))

for data in data_list:
    link = (data.find_element(By.TAG_NAME,"a").get_attribute("href"))
    product_link.append(link)
    re = Request(url=link, headers={"User-Agent":generate_user_agent()})
    rel = requests.get(link)
    page = urlopen(re)
    soup = BeautifulSoup(page, "html.parser")
    product_names.append(soup.find(id="notexist_title").string)
    
    script_org = soup.find_all("script")
    script = script_org[-1].get_text()#
    script_low = (script[script.find("product.state = {")+len("product.state = {")-1:script.rfind(";")+1])
    val = eval(script_low[:script_low.rfind("}")+1].replace("false","False").replace("null","None").replace("true","True"))
    try:
        product_sell.append(val["differenceColorGoods"][0]["saleRate"])
    except IndexError:
        product_sell.append(None)
    product_numbers.append(val["goodsNo"])
    product_brand.append(val["brand"])
    product_gender.append(val["sex"])
    product_bigCategory.append(val["category"]["categoryDepth1Title"])
    product_smallCategory.append(val["category"]["categoryDepth2Title"])
    product_review_count.append(val["goodsReview"]["totalCount"])
    product_member_price.append(val["goodsPriceMemberLevels"][0]["price"])
    product_nomal_price.append(val["goodsPrice"]["originPrice"])
    product_tags.append(str(val["goodsTags"]))

In [67]:
import requests
c=0
# 사용자 정의 User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for goods_code in product_numbers:
    # URL을 동적으로 생성
    url = f'https://www.musinsa.com/app/product/goodsview_stats/{goods_code}/0?loc=goods_rank&menu=view'

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP 요청이 성공적인지 확인

        text_data = response.text  # 응답 텍스트 가져오기

        # 'total' 다음에 나오는 값이 ','까지의 숫자로 이루어져 있다고 가정
        total_index = text_data.index('total')
        comma_index = text_data.index(',', total_index)

        total_value = text_data[total_index + len('total'):comma_index+2]

        print(f"{product_names[c]}:의 총 판매수는: {total_value}")
        c+=1
    except requests.exceptions.RequestException as e:
        print(f"Error during request for goods code {goods_code}: {e}")


프론트 워싱 와이드 데님 팬츠 (NORMAL BLACK):의 총 판매수는: ":7454,"
와이드 데님 팬츠 [그레이]:의 총 판매수는: ":11487,"
프론트 워싱 와이드 데님 팬츠 (YELLOW BLACK):의 총 판매수는: ":25059,"
파라슈트 카고 리벳 데님 팬츠 (BLACK):의 총 판매수는: ":4093,"
바이오스톤 워싱 와이드 데님 팬츠 (DEEP GRAY):의 총 판매수는: ":11701,"
원턱 파라슈트 카고 데님팬츠_그레이:의 총 판매수는: ":12865,"
KIPAS 스트레이트 데님 팬츠 [스카이 블루]:의 총 판매수는: ":0,"
프론트 워싱 와이드 데님 팬츠 (DEEP GREEN):의 총 판매수는: ":4174,"
옐로우 워싱 와이드 데님팬츠 인디고 SOLP2503:의 총 판매수는: ":3723,"
데저트 워싱 와이드 데님팬츠 블랙 SOLP2501:의 총 판매수는: ":1332,"
투 턱 와이드 데님 팬츠 [로우 블랙]:의 총 판매수는: ":1741,"
(기모옵션추가) 이지 와이드 데님 팬츠 빈티지 워시드 블루:의 총 판매수는: ":11397,"
와이드 데님 팬츠 [블랙]:의 총 판매수는: ":5025,"
바이오스톤 워싱 와이드 데님 팬츠 (CREAM):의 총 판매수는: ":19140,"
논페이드 와이드 데님팬츠[딥블루인디고]:의 총 판매수는: ":1413,"


KeyboardInterrupt: 

In [31]:
df = pd.DataFrame()
df["링크"] = product_link
df["이름"] = product_names
df["브랜드"] = product_brand
df["성별"] = product_gender
df["누적판매량"] = product_sell
df["대분류"] = product_bigCategory
df["중분류"] = product_smallCategory
df["좋아요수"] = product_goods
df["리뷰개수"] = product_review_count
df["회원가"] = product_member_price
df["일반가"] = product_nomal_price
df["태그"] = product_tags
df["상품번호"]=product_number

In [32]:
df

,링크,이름,브랜드,성별,누적판매량,대분류,중분류,좋아요수,리뷰개수,회원가,일반가,태그,상품번호
0,https://www.musinsa.com/app/goods/3124492?loc=...,프론트 워싱 와이드 데님 팬츠 (NORMAL BLACK),trillion,"[남성, 여성]",44.0,바지,데님 팬츠,"15,350",1416,29800,53000,"['와이드데님', '루즈핏', '남자청바지', '팬츠', '데님', '청바지', '...",3124492
1,https://www.musinsa.com/app/goods/3467143?loc=...,와이드 데님 팬츠 [미디엄 블루],againstallodds,"[남성, 여성]",50.0,바지,데님 팬츠,"32,644",2080,31320,69800,"['데님', '청바지', '와이드팬츠', '데님팬츠', '와이드데님', '23F\\...",3467143
2,https://www.musinsa.com/app/goods/3467190?loc=...,와이드 데님 팬츠 [그레이],againstallodds,"[남성, 여성]",50.0,바지,데님 팬츠,"23,994",1038,31320,69800,"['와이드팬츠', '데님', '청바지', '와이드데님', '데님팬츠', '23F\\...",3467190
3,https://www.musinsa.com/app/goods/2028329?loc=...,스트레이트 데님 팬츠 [워시드 블랙],musinsastandard,[남성],20.0,바지,데님 팬츠,"29,540",10115,39491,49900,"['일자데님', '레귤러핏', '데님컬렉션', '플러스배송', '데님팬츠맨', '데...",2028329
4,https://www.musinsa.com/app/goods/3124490?loc=...,프론트 워싱 와이드 데님 팬츠 (YELLOW BLACK),trillion,"[남성, 여성]",25.0,바지,데님 팬츠,"54,637",5362,35382,53000,"['와이드데님', '루즈핏', '남자청바지', '청바지', '흑청', '데님와이드핏...",3124490
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,https://www.musinsa.com/app/goods/2391744?loc=...,스트레이트 데님 팬츠 [스카이 블루],musinsastandard,[남성],20.0,바지,데님 팬츠,"8,861",2341,39491,49900,"['일자데님', '레귤러핏', '데님컬렉션', 'PLUS배송', '플러스배송', '...",2391744
86,https://www.musinsa.com/app/goods/2494147?loc=...,바이오스톤 워싱 와이드 데님 팬츠 (BLACK),trillion,"[남성, 여성]",26.0,바지,데님 팬츠,"9,204",2001,35382,54000,"['스트레이트', '청바지', '워싱', '데님', '남자', '블랙진', '흑청'...",2494147
87,https://www.musinsa.com/app/goods/3613296?loc=...,논페이드 와이드 데님팬츠[딥블루인디고],chambros,"[남성, 여성]",42.0,바지,데님 팬츠,"2,875",236,39760,69000,"['데님바지', '데님와이드핏', '와이드데님', '데님팬츠', '여자청바지', '...",3613296
88,https://www.musinsa.com/app/goods/2322316?loc=...,[-6cm 숏버전 추가] M160 와이드 데님 팬츠_진청,waar,[남성],60.0,바지,데님 팬츠,"2,546",600,35900,89000,"['와이드청바지', '와이드팬츠', '바지', '데님', '청바지', '남자청바지'...",2322316


In [19]:
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.216)
Stacktrace:
#0 0x56304ac08f83 <unknown>
#1 0x56304a8c1cf7 <unknown>
#2 0x56304a8a4ff5 <unknown>
#3 0x56304a8a618a <unknown>
#4 0x56304a8c2257 <unknown>
#5 0x56304a89506e <unknown>
#6 0x56304a94901f <unknown>
#7 0x56304a93b078 <unknown>
#8 0x56304a902dd4 <unknown>
#9 0x56304a9041de <unknown>
#10 0x56304abcd531 <unknown>
#11 0x56304abd1455 <unknown>
#12 0x56304abb9f55 <unknown>
#13 0x56304abd20ef <unknown>
#14 0x56304ab9d99f <unknown>
#15 0x56304abf6008 <unknown>
#16 0x56304abf61d7 <unknown>
#17 0x56304ac08124 <unknown>
#18 0x7fa3b0e94ac3 <unknown>
